In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/17"
pmc = "21018005" # 店舗ID
SPREADSHEET_KEY = '17sHb0RhOTLQKkOhbLxHSjV_6iW6hByiZ4I1iAFSuD08' # スプレッドシートのID
pstart = 481 # パチンコの1台目
sstart = 1 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+123@gmail.com"
user_pw = "paruparu21"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/17', '025834', '481', '37692.0', '13242.0', '25', '7']
['08/17', '025834', '482', '35128.0', '5719.0', '19', '6']
['08/17', '025834', '483', '14359.0', '302.0', '7', '2']
['08/17', '025834', '484', '34359.0', '5418.0', '19', '5']
['08/17', '025834', '485', '35897.0', '-12638.0', '9', '4']
['08/17', '025834', '486', '35641.0', '12339.0', '24', '4']
['08/17', '025834', '487', '26154.0', '-7823.0', '7', '4']
['08/17', '025834', '488', '31538.0', '23774.0', '29', '6']
['08/17', '025834', '489', '27692.0', '3612.0', '15', '5']
['08/17', '025834', '490', '31795.0', '-10832.0', '8', '4']
['08/17', '025834', '491', '48718.0', '-17453.0', '12', '5']
['08/17', '025834', '492', '15385.0', '-12036.0', '0', '0']
['08/17', '025834', '493', '37692.0', '3010.0', '19', '7']
['08/17', '025834', '494', '30769.0', '39121.0', '39', '4']
['08/17', '025834', '495', '20769.0', '-9930.0', '4', '2']
['08/17', '025834', '496', '39744.0', '-15948.0', '9', '3']
['08/17', '025834', '497', '48462.0', '-13240.0'

['08/17', '025553', '620', '39231.0', '603.0', '27', '9']
['08/17', '025553', '621', '16154.0', '-6319.0', '8', '4']
['08/17', '025553', '622', '14615.0', '3311.0', '12', '3']
['08/17', '025553', '623', '6923.0', '3913.0', '9', '1']
['08/17', '025553', '624', '13333.0', '-9629.0', '1', '1']
['08/17', '025665', '625', '44872.0', '-18356.0', '15', '8']
['08/17', '025665', '626', '49744.0', '-20763.0', '16', '5']
['08/17', '025665', '627', '41282.0', '-29490.0', '6', '4']
['08/17', '025665', '628', '39487.0', '8728.0', '28', '5']
['08/17', '025665', '629', '55385.0', '-3610.0', '31', '10']
['08/17', '025665', '630', '46667.0', '5418.0', '31', '7']
['08/17', '025665', '631', '36154.0', '16251.0', '30', '3']
['08/17', '025665', '632', '39231.0', '-12939.0', '14', '4']
['08/17', '025665', '633', '45897.0', '-25277.0', '11', '6']
['08/17', '025665', '634', '46923.0', '-29791.0', '9', '7']
['08/17', '025665', '635', '49744.0', '-14744.0', '19', '6']
['08/17', '025665', '636', '40000.0', '21066

['08/17', '025662', '767', '7436.0', '5418.0', '12', '2']
['08/17', '025662', '768', '1282.0', '-1203.0', '0', '0']
['08/17', '025849', '769', '3333.0', '-1504.0', '1', '1']
['08/17', '025849', '770', '6154.0', '-5115.0', '0', '0']
['08/17', '025849', '771', '3077.0', '-1203.0', '1', '1']
['08/17', '025849', '772', '2564.0', '-2106.0', '0', '0']
['08/17', '025849', '773', '1538.0', '-1203.0', '0', '0']
['08/17', '025849', '774', '3077.0', '-1203.0', '1', '1']
['08/17', '025677', '775', '18462.0', '4214.0', '15', '5']
['08/17', '025677', '776', '17436.0', '17756.0', '15', '4']
['08/17', '025677', '777', '9487.0', '7524.0', '11', '2']
['08/17', '025677', '778', '31026.0', '-10231.0', '11', '4']
['08/17', '025677', '779', '9487.0', '-8124.0', '0', '0']
['08/17', '025677', '780', '21026.0', '-13842.0', '3', '2']
['08/17', '025677', '781', '29744.0', '-3911.0', '15', '7']
['08/17', '025677', '782', '29231.0', '-20763.0', '1', '1']
['08/17', '025677', '783', '7949.0', '-5416.0', '1', '1']
['

['08/17', '025842', '910', '10000.0', '10834.0', '14', '3']
['08/17', '025842', '911', '4103.0', '-3610.0', '0', '0']
['08/17', '025842', '912', '6923.0', '-6018.0', '0', '0']
['08/17', '025521', '913', '27436.0', '-5717.0', '21', '9']
['08/17', '025521', '914', '2564.0', '1806.0', '5', '2']
['08/17', '025521', '915', '3077.0', '-902.0', '2', '1']
['08/17', '025521', '916', '28974.0', '904.0', '29', '12']
['08/17', '025863', '917', '8462.0', '-2707.0', '9', '3']
['08/17', '025863', '918', '11538.0', '603.0', '20', '3']
['08/17', '025863', '919', '12821.0', '6922.0', '31', '5']
['08/17', '025877', '920', '17949.0', '1806.0', '21', '8']
['08/17', '025877', '921', '19744.0', '-1504.0', '21', '8']
['08/17', '025877', '922', '20256.0', '-6018.0', '11', '8']
['08/17', '025877', '923', '35128.0', '-3008.0', '32', '19']
['08/17', '025877', '924', '25641.0', '4515.0', '33', '12']
['08/17', '025877', '925', '33077.0', '-2707.0', '32', '14']
['08/17', '025877', '926', '20000.0', '2107.0', '21', '

['08/17', '025852', '1050', '14615.0', '24376.0', '73', '4']
['08/17', '025838', '1051', '23333.0', '-1203.0', '16', '5']
['08/17', '025838', '1052', '24103.0', '-2106.0', '16', '5']
['08/17', '025838', '1053', '14103.0', '13844.0', '19', '5']
['08/17', '025838', '1054', '14359.0', '-7522.0', '4', '2']
['08/17', '025568', '1055', '19744.0', '-14143.0', '1', '1']
['08/17', '025568', '1056', '14872.0', '28589.0', '32', '2']
['08/17', '025851', '1057', '21282.0', '16251.0', '98', '9']
['08/17', '025851', '1058', '14359.0', '-5416.0', '18', '4']
['08/17', '025235', '1059', '21538.0', '1806.0', '27', '5']
['08/17', '025235', '1060', '14872.0', '-3008.0', '13', '4']
['08/17', '025235', '1061', '20513.0', '3612.0', '33', '10']
['08/17', '025784', '1062', '38718.0', '-2106.0', '99', '20']
['08/17', '025785', '1063', '35128.0', '-3008.0', '31', '31']
['08/17', '025831', '1064', '34359.0', '2408.0', '42', '13']
['08/17', '025855', '1065', '28205.0', '-7522.0', '36', '19']
['08/17', '025711', '10

['08/17', '025777', '1187', '15897.0', '-5717.0', '12', '12']
['08/17', '025626', '1188', '22564.0', '-6018.0', '25', '25']
['08/17', '025626', '1189', '24103.0', '3311.0', '44', '44']
['08/17', '025626', '1190', '24359.0', '1205.0', '39', '39']
['08/17', '025139', '1191', '32564.0', '8126.0', '53', '13']
['08/17', '025139', '1192', '28718.0', '6320.0', '39', '7']
['08/17', '025728', '1193', '15897.0', '-6619.0', '28', '8']
['08/17', '025705', '1195', '34359.0', '-8124.0', '30', '16']
['08/17', '025759', '1196', '22821.0', '2408.0', '29', '10']
['08/17', '025776', '1197', '24872.0', '-10832.0', '22', '6']
['08/17', '025780', '1198', '31282.0', '-6920.0', '42', '13']
['08/17', '025613', '1199', '33590.0', '12941.0', '41', '41']
['08/17', '025466', '1200', '31282.0', '-10832.0', '16', '7']
['08/17', '120010', '1', '1965', '-185.0', '6', '4', '--']
['08/17', '120010', '2', '2369', '-833.0', '7', '4', '--']
['08/17', '120010', '3', '6129', '185.0', '22', '20', '--']
['08/17', '120010', '4'

['08/17', '119916', '124', '2218', '-231.0', '9', '3', '--']
['08/17', '119916', '125', '8550', '185.0', '27', '42', '--']
['08/17', '119916', '126', '4763', '1620.0', '24', '16', '--']
['08/17', '119916', '127', '2602', '93.0', '10', '4', '--']
['08/17', '119916', '128', '7600', '1111.0', '30', '25', '--']
['08/17', '120096', '129', '3241', '-417.0', '11', '8', '--']
['08/17', '120096', '130', '1136', '-417.0', '2', '4', '--']
['08/17', '120096', '131', '1260', '-694.0', '2', '3', '--']
['08/17', '120096', '132', '1062', '-509.0', '1', '4', '--']
['08/17', '120096', '133', '480', '-278.0', '1', '0', '--']
['08/17', '120096', '134', '1153', '-139.0', '4', '2', '--']
['08/17', '120096', '135', '1858', '-648.0', '4', '7', '--']
['08/17', '120096', '136', '4563', '2037.0', '24', '20', '--']
['08/17', '120096', '137', '209', '-278.0', '0', '0', '--']
['08/17', '120096', '138', '6889', '-926.0', '22', '23', '--']
['08/17', '120096', '139', '1100', '-139.0', '3', '2', '--']
['08/17', '120096

['08/17', '120108', '260', '3095', '-324.0', '0', '0', '19']
['08/17', '120108', '261', '3402', '1435.0', '0', '0', '23']
['08/17', '120108', '262', '5247', '926.0', '0', '0', '36']
['08/17', '120025', '263', '1576', '-787.0', '4', '1', '--']
['08/17', '120025', '264', '5745', '2176.0', '29', '15', '--']
['08/17', '120025', '265', '5139', '463.0', '18', '18', '--']
['08/17', '120025', '266', '498', '-324.0', '1', '0', '--']
['08/17', '120025', '267', '1268', '139.0', '5', '2', '--']
['08/17', '120025', '268', '4564', '2500.0', '23', '17', '--']
['08/17', '120025', '269', '3434', '648.0', '16', '4', '--']
['08/17', '120025', '270', '1913', '185.0', '9', '2', '--']
['08/17', '120025', '271', '1110', '93.0', '5', '1', '--']
['08/17', '120025', '272', '2614', '926.0', '14', '4', '--']
['08/17', '120025', '273', '4024', '648.0', '15', '10', '--']
['08/17', '120025', '274', '968', '-185.0', '2', '2', '--']
['08/17', '120025', '275', '7545', '1991.0', '31', '25', '--']
['08/17', '120025', '27

['08/17', '120138', '393', '1367', '648.0', '0', '1', '5']
['08/17', '120138', '394', '0', '0', '0', '0', '--']
['08/17', '120138', '395', '0', '0', '0', '0', '--']
['08/17', '120138', '396', '0', '0', '0', '0', '--']
['08/17', '120129', '397', '3149', '-2037.0', '0', '8', '11']
['08/17', '120129', '398', '2359', '-833.0', '0', '8', '11']
['08/17', '120129', '399', '2994', '-1111.0', '0', '11', '15']
['08/17', '120129', '400', '3282', '1852.0', '0', '8', '34']
['08/17', '120129', '401', '4658', '3843.0', '0', '14', '48']
['08/17', '120129', '402', '4839', '324.0', '0', '19', '33']
['08/17', '120130', '403', '4209', '-1944.0', '0', '9', '12']
['08/17', '120130', '404', '4542', '-2685.0', '0', '6', '13']
['08/17', '120130', '405', '5850', '-2917.0', '0', '9', '22']
['08/17', '120130', '406', '5022', '139.0', '0', '8', '26']
['08/17', '120130', '407', '5474', '-1065.0', '0', '9', '21']
['08/17', '120130', '408', '4022', '-2685.0', '0', '6', '14']
['08/17', '120122', '409', '6084', '-648.0

{'spreadsheetId': '17sHb0RhOTLQKkOhbLxHSjV_6iW6hByiZ4I1iAFSuD08',
 'updates': {'spreadsheetId': '17sHb0RhOTLQKkOhbLxHSjV_6iW6hByiZ4I1iAFSuD08',
  'updatedRange': 'S20RAW!F88559:M89038',
  'updatedRows': 480,
  'updatedColumns': 8,
  'updatedCells': 3840}}